In [10]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv('heart_attack_dataset.csv')

In [5]:
# print(df.shape)
# print(df.head())
# categorization of columns for reference
BinaryColumns = ['Smoker', 'Diabetes', 'Hypertension', 'FamilyHistory', 
                'AlcoholConsumption', 'Medication', 'ExerciseInducedAngina', 
                 'PreviousHeartAttack', 'StrokeHistory']
    
ContinuousColumns = ['Age', 'Cholesterol', 'BloodPressure', 'HeartRate', 'BMI', 'PhysicalActivity', 
                     'Income', 'MaxHeartRate', 'ST_Depression', 'NumberOfMajorVessels']
    
CategoricalColumns = ['Gender', 'Diet', 'StressLevel', 'Ethnicity', 
                      'EducationLevel', 'ChestPainType', 'ECGResults', 
                      'Slope', 'Thalassemia', 'Residence', 'EmploymentStatus', 
                      'MaritalStatus']

In [26]:
# extract features and target
X = df[['Smoker', 'Diabetes', 'Hypertension']].values
y = df['Outcome'].values

# convert 'heartk attack' and 'not heart attack' to binary output
y = (y == 'Heart Attack').astype(int)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# standardize inputs
scaler = StandardScaler()
X_train = torch.tensor(X_train, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.float32).view(-1, 1)

X_test = torch.tensor(X_test, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.float32).view(-1, 1)

In [27]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(3, 10) # input layer
        self.fc2 = nn.Linear(10, 5) # hidden layer 1
        self.fc3 = nn.Linear(5, 1)  # hidden layer 2 to output layer
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))   # ReLu on hidden layer 1
        x = torch.relu(self.fc2(x))   # ReLu on hidden layer 2
        x = self.sigmoid(self.fc3(x)) # sigmoid activiation on output
        return x

In [28]:
# hyper-parameters
learn_rate = 0.01
epochs = 100

In [29]:
# the model defined above
model = NN()

# loss function 
criterion = nn.BCELoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr = learn_rate)

In [30]:
# training
for epoch in range(epochs):
    # forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch[{epoch + 1} / {epochs}], Loss: {loss.item():.4f}')
print("Training Complete")

Epoch[10 / 100], Loss: 0.6937
Epoch[20 / 100], Loss: 0.6932
Epoch[30 / 100], Loss: 0.6932
Epoch[40 / 100], Loss: 0.6932
Epoch[50 / 100], Loss: 0.6932
Epoch[60 / 100], Loss: 0.6931
Epoch[70 / 100], Loss: 0.6931
Epoch[80 / 100], Loss: 0.6931
Epoch[90 / 100], Loss: 0.6931
Epoch[100 / 100], Loss: 0.6931
Training Complete


In [32]:
# set evaluation mode
model.eval()

# disable gradient computation for eval
with torch.no_grad():
    test_outputs = model(X_test)
    test_outputs = (test_outputs > 0.5).float()
    # convert probabilities to binary prediction as we did to heart attack/no heart attack
    
    # calculate accuracy
    accuracy = (test_outputs == y_test).float().mean()
    print(f'Test Accuracy: {accuracy.item() * 100:.2f}%')

Test Accuracy: 49.80%
